In [1]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [2]:
# small helper function to take in true/false samples and get back +-1 eigenvalue
def get_eigenvals_from_tf(sample_list):
    result = 1 - 2 * sample_list.astype(int)
    return result

# get X syndromes from measuring the entangled |+> state of auxilary qubits in the z basis.
"""
eigenvalues is a list of +-1's for each qubit, stabilizers is a list of lists of ints, where each one represents
an index where the stabilizer operator is applied (e.g. X_1 X_3 X_5 X_7, X_4 X_5 X_6 X_7, X_2 X_3 X_6 X_7 becomes 
[[0,2,4,6],[3,4,5,6],[1,2,5,6]] (0-indexed)
Output is a list of +-1's, one for each stabilizer.
"""
def get_syndromes(eigenvalues, stabilizers):
    syndrome = []
    for stab in stabilizers:
        eigval = 1
        for idx in stab:
            eigval *= eigenvalues[idx]
        syndrome.append(eigval)
    return syndrome


""" This function takes 4 arguments:
    1. qubit_count, integer number of qubits
    2. syndrome, list of +-1's
    3. syndrome_table, a dictionary where keys are the error syndromes (lists of +-1's), and the values are the qubits to apply gates to fix
    4. gate_type, a character determining the gate to apply, either X or Z for the 7 color code.
"""
def create_subcircuit_for_syndrome_based_correction(qubit_count, syndrome, syndrome_table, gate_type):
    @squin.kernel
    def error_correction_subcirc():
        q = squin.qalloc(qubit_count)  # allocate qubits
        if gate_type == 'X':
            squin.x(syndrome_table[syndrome])
        if gate_type == 'Z':
            squin.z(syndrome_table[syndrome])    
    return error_correction_subcirc
    
    

In [ ]:
# This is the syndrome to correction table for the 7 color code. It is symmtric in X and Z so it works for both.
syndrome_table = {[-1,1,1]:0, [1,1,-1]:1, [-1,1,-1]:2, [1,-1,1]:3, [-1,-1,1]:4, [1,-1,-1]:5, [-1,-1,-1]:6}

In [ ]:
# Here are some tests, using the MSD circuit with no noise (we should always get syndrome [1,1,1]

# Here are some tests, using the MSD circuit with a randomtime based error after the fact.